<a href="https://colab.research.google.com/github/IvanLudvig/eggcounting/blob/master/eggcounting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import json
from datetime import datetime

In [2]:
with open('/content/drive/MyDrive/eggcounting/data0.json') as f:
    data = json.load(f)['messages']
    df = pd.DataFrame(data)

In [3]:
df

,id,type,timestamp,timestampEdited,callEndedTimestamp,isPinned,content,author,attachments,embeds,reactions,mentions,reference
0,886460807087550536,Default,2021-09-12T03:58:46.055+00:00,None,None,False,1,"{'id': '788951038971215924', 'name': 'thelegen...",[],[],[],[],NaN
1,886460808425537597,Default,2021-09-12T03:58:46.374+00:00,None,None,False,2,"{'id': '788951038971215924', 'name': 'thelegen...",[],[],[],[],NaN
2,886460810455580732,Default,2021-09-12T03:58:46.858+00:00,None,None,False,3,"{'id': '788951038971215924', 'name': 'thelegen...",[],[],[],[],NaN
3,886460811722227713,Default,2021-09-12T03:58:47.16+00:00,None,None,False,4,"{'id': '788951038971215924', 'name': 'thelegen...",[],[],[],[],NaN
4,886460813790023730,Default,2021-09-12T03:58:47.653+00:00,None,None,False,5,"{'id': '788951038971215924', 'name': 'thelegen...",[],[],[],[],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58943,893185301395148881,Default,2021-09-30T17:19:30.433+00:00,None,None,False,58876,"{'id': '690314276963090532', 'name': 'nodu', '...",[],[],[],[],NaN
58944,893185305639784469,Default,2021-09-30T17:19:31.445+00:00,None,None,False,58877,"{'id': '690314276963090532', 'name': 'nodu', '...",[],[],[],[],NaN
58945,893185310219964506,Default,2021-09-30T17:19:32.537+00:00,None,None,False,58878,"{'id': '690314276963090532', 'name': 'nodu', '...",[],[],[],[],NaN
58946,893185314535915550,Default,2021-09-30T17:19:33.566+00:00,None,None,False,58879,"{'id': '690314276963090532', 'name': 'nodu', '...",[],[],[],[],NaN


In [4]:
df['user'] = df['author'].map(lambda x: x['name'])

In [5]:
df = df[['id', 'timestamp', 'content', 'user']]

filter only valid counts

In [6]:
df = df[df['content'].str.isnumeric()]

# Total counts

In [7]:
counts_df = df['user'].value_counts(sort=True).reset_index()
counts_df.columns = ['user', 'counts']

In [8]:
counts_df

,user,counts
0,Whit4You,22955
1,brainlet,19926
2,const,3892
3,thelegendary08,3605
4,Pranoodles🍜 (she/her),2505
5,ob69jl,1715
6,striker,1571
7,FortniteDude70,658
8,core,511
9,Dawidluna,312


# 1K days

In [9]:
df.head()

,id,timestamp,content,user
0,886460807087550536,2021-09-12T03:58:46.055+00:00,1,thelegendary08
1,886460808425537597,2021-09-12T03:58:46.374+00:00,2,thelegendary08
2,886460810455580732,2021-09-12T03:58:46.858+00:00,3,thelegendary08
3,886460811722227713,2021-09-12T03:58:47.16+00:00,4,thelegendary08
4,886460813790023730,2021-09-12T03:58:47.653+00:00,5,thelegendary08


In [10]:
df['date'] = df['timestamp'].apply(lambda ts: ts.split('T')[0])

In [11]:
days_df = df[['user', 'date']]
days_df.head()

,user,date
0,thelegendary08,2021-09-12
1,thelegendary08,2021-09-12
2,thelegendary08,2021-09-12
3,thelegendary08,2021-09-12
4,thelegendary08,2021-09-12


In [12]:
group = days_df.groupby('user')
group = group.apply(lambda x: x['date'].to_numpy()).reset_index()
group.columns = ['user', 'dates']
group.head()

,user,dates
0,0-0,"[2021-09-12, 2021-09-12, 2021-09-12, 2021-09-1..."
1,Brendan7625,"[2021-09-18, 2021-09-18, 2021-09-18, 2021-09-1..."
2,Dawidluna,"[2021-09-16, 2021-09-16, 2021-09-16, 2021-09-1..."
3,FortniteDude70,"[2021-09-12, 2021-09-12, 2021-09-12, 2021-09-1..."
4,GermanMgr14 [ARG],[2021-09-18]


In [13]:
for date in days_df['date'].unique():
    group[date] = group['dates'].map(lambda x: (x == date).sum())

In [14]:
group = group.drop(columns=['dates'])
group.head()

,user,2021-09-12,2021-09-13,2021-09-14,2021-09-15,2021-09-16,2021-09-17,2021-09-18,2021-09-19,2021-09-20,2021-09-21,2021-09-22,2021-09-23,2021-09-24,2021-09-25,2021-09-26,2021-09-27,2021-09-28,2021-09-29,2021-09-30
0,0-0,5,47,0,0,0,0,0,42,8,0,0,0,0,0,1,0,0,0,0
1,Brendan7625,0,0,0,0,0,0,100,0,0,0,0,0,0,0,0,0,0,0,0
2,Dawidluna,0,0,0,0,200,80,0,0,0,32,0,0,0,0,0,0,0,0,0
3,FortniteDude70,115,180,0,0,0,0,0,0,0,0,0,0,0,204,124,5,10,10,10
4,GermanMgr14 [ARG],0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [15]:
counts = group.drop(columns=['user']).values

In [16]:
group['counts_list'] = counts.tolist()
group = group[['user', 'counts_list']]
group.head()

,user,counts_list
0,0-0,"[5, 47, 0, 0, 0, 0, 0, 42, 8, 0, 0, 0, 0, 0, 1..."
1,Brendan7625,"[0, 0, 0, 0, 0, 0, 100, 0, 0, 0, 0, 0, 0, 0, 0..."
2,Dawidluna,"[0, 0, 0, 0, 200, 80, 0, 0, 0, 32, 0, 0, 0, 0,..."
3,FortniteDude70,"[115, 180, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20..."
4,GermanMgr14 [ARG],"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."


### Total 1k days

In [17]:
group['1k_days'] = group['counts_list'].map(lambda x: (np.array(x) >= 1000).sum())
total_1kdays = group[['user', '1k_days']].sort_values(by='1k_days', ascending=False).reset_index(drop=True)
total_1kdays = total_1kdays[total_1kdays['1k_days']>0]

In [18]:
total_1kdays

,user,1k_days
0,Whit4You,8
1,brainlet,7
2,ob69jl,1
3,const,1


## Max 1k streak

In [19]:
def get_max_streak(counts_list):
    bool_array = np.array(counts_list) >= 1000
    bool_array = np.split(bool_array, np.where(np.diff(bool_array) != 0)[0]+1)
    bool_array = list(filter(lambda x: x[0], bool_array))
    bool_array = [x.shape[0] for x in bool_array]
    return max(bool_array) if len(bool_array) > 0 else 0

In [20]:
group['max_streak'] = group['counts_list'].map(get_max_streak)

In [21]:
max_streak = group[['user', 'max_streak']].sort_values(by='max_streak', ascending=False).reset_index(drop=True)
max_streak = max_streak[max_streak['max_streak']>0]

In [22]:
max_streak

,user,max_streak
0,brainlet,6
1,Whit4You,3
2,ob69jl,1
3,const,1
